In [1]:
import pandas as pd
import gc
import polars as pl
from tqdm.auto import tqdm
from scipy.optimize import curve_fit
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn import set_config
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from utils import *

set_config(transform_output="pandas")
pl.enable_string_cache()

## cargamos los datos
lazy_wide_df = pl.scan_parquet("../data/wide.parquet")
#print(lazy_wide_df.collect_schema())

lazy_test_df = pl.scan_parquet("../data/ids_test_expanded.parquet")


In [2]:
# ==========================================
# CONFIGURACIÓN DE MODELOS Y PREPROCESSORS
# ==========================================

# Configuración del preprocessor
preprocessor_region = ColumnTransformer(
    transformers=[
        (
            "onehot",
            OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
            ["store_type", "brand_house", "cluster"],
        ),
        (
            "ordinal",
            OrdinalEncoder(categories=[["Primavera", "Verano", "Otoño", "Invierno"]]),
            ["estacion"],
        ),
        (
            "scaler",
            StandardScaler(),
            [   # "price",
                "residuos_precio",
                "loyalty_count", 
                "non_loyalty_count",
                "year",
                "quantity",
                "days_since_start",
                "avg_price_ratio",
                "avg_profit_ratio",
            ],
        ),
    ],
    remainder="passthrough",
)

# Pipeline optimizado
pipeline_region = make_pipeline(
    preprocessor_region,
    HistGradientBoostingRegressor(
        max_iter=200,
        max_depth=6,
        learning_rate=0.05,
        min_samples_leaf=15,
        validation_fraction=0.2,
        early_stopping=True,
        random_state=42,
    ),
)

# ==========================================
# PREPARACIÓN DE DATOS AGRUPADOS
# ==========================================

# Lazy grouped corregido para región
lazy_grouped_region = (lazy_wide_df
    # lazy_wide_df.with_columns(
    #     # (
    #     #     (pl.col("date") - pl.col("date").min()).cast(pl.Int64) / 1000 / 60 / 60 / 24
    #     # ).alias("days_since_start")
    # )
    .group_by(
        [
            "subgroup", "region", "store_id", "cluster", "store_type",
            "estacion", "day", "days_since_start", "weekday", "month", 
            "year", "brand_house", "quantity",
        ] #CHECK QUANTITY --> NO SE SI TIENE QUE ESTAR ACA O NO
    )
    .agg(
        [
            pl.col("price").mean().alias("price"),
            pl.col("total_sales").sum().alias("total_sales"),
            pl.col("day_sin").mean().alias("day_sin"),
            pl.col("day_cos").mean().alias("day_cos"),
            pl.col("weekday_sin").mean().alias("weekday_sin"),
            pl.col("weekday_cos").mean().alias("weekday_cos"),
            pl.col("month_sin").mean().alias("month_sin"),
            pl.col("month_cos").mean().alias("month_cos"),
            pl.col("loyalty_count").mean().alias("loyalty_count"),
            pl.col("non_loyalty_count").mean().alias("non_loyalty_count"),
            pl.col("base_price").mean().alias("base_price"),
            pl.col("costos").mean().alias("costos"),
            # pl.col("quantity").mean().alias("quantity"),
        ]
    )
    .with_columns(
        (pl.col("price") / pl.col("base_price")).alias("avg_price_ratio"),
        (pl.col("price") / pl.col("costos")).alias("avg_profit_ratio"),
    )
)

In [6]:
# ==========================================
# LOOP PRINCIPAL CON POLARS - OPTIMIZADO
# ==========================================

# Configuración
LIMIT_PRODUCTS = None
LIMIT_REGIONS = None
USE_REGIONS = False

# Variables para resultados
dict_results = {}

# Obtener listas de productos y regiones
productos = lazy_test_df.select("subgroup").unique().collect().to_pandas()["subgroup"].values
regiones = lazy_test_df.select("region").unique().collect().to_pandas()["region"].values

# Aplicar límites si están configurados
if LIMIT_PRODUCTS:
    productos = productos[:LIMIT_PRODUCTS]
if LIMIT_REGIONS:
    regiones = regiones[:LIMIT_REGIONS]

# Configurar procesamiento según USE_REGIONS
if USE_REGIONS:
    print("📊 PROCESANDO CON REGIONES SEPARADAS")
    total_combinations = len(productos) * len(regiones)
    combinations = [(prod, region) for prod in productos for region in regiones]
else:
    print("📊 PROCESANDO SIN FILTRAR POR REGIÓN")
    total_combinations = len(productos)
    combinations = [(prod, None) for prod in productos]

print("📊 CONFIGURACIÓN DE PROCESAMIENTO:")
print(f"   - Productos: {len(productos)}")
print(f"   - Regiones: {len(regiones) if USE_REGIONS else 'Todas juntas'}")
print(f"   - Total combinaciones: {total_combinations}")
print(f"   - Memoria inicial: {check_memory():.1f} MB")
print("=" * 50)

# Procesamiento optimizado
count = 0

for prod, region in tqdm(combinations, desc="Procesando combinaciones"):
    count += 1
    progress = (count / total_combinations) * 100
    
    print(f"\n🎯 PRODUCTO: {prod}")
    
    if USE_REGIONS and region:
        print(f"   🌍 Región: {region} ({count}/{total_combinations} - {progress:.1f}%)")
    else:
        print(f"   🌐 Procesando todas las regiones juntas ({count}/{total_combinations} - {progress:.1f}%)")
    
    # Caso especial: Basketball
    if prod == "Basketball":
        print("   📝 Caso especial: asignando ventas = 0")
        basketball_results = process_basketball_special_case(lazy_test_df, prod)
        dict_results.update(basketball_results)
        continue
    
    try:
        # Procesar combinación producto-región
        region_results = process_product_region_combination_polars(
            prod, region, lazy_grouped_region, lazy_wide_df, lazy_test_df, pipeline_region
        )
        
        # Agregar resultados
        dict_results.update(region_results)
        
    except Exception as e:
        print(f"   ❌ Error en {prod}-{region}: {e}")
        continue
    
    # Limpieza de memoria
    if count % 5 == 0:
        current_memory = check_memory()
        gc.collect()
        new_memory = check_memory()
        print(f"   🧹 Memoria: {current_memory:.1f} → {new_memory:.1f} MB (liberados: {current_memory-new_memory:.1f} MB)")

print("\n" + "=" * 50)
print(f"✅ PROCESAMIENTO COMPLETADO")
print(f"   - Total predicciones generadas: {len(dict_results):,}")
print(f"   - Combinaciones procesadas: {count}/{total_combinations}")
print("=" * 50)

📊 PROCESANDO SIN FILTRAR POR REGIÓN
📊 CONFIGURACIÓN DE PROCESAMIENTO:
   - Productos: 74
   - Regiones: Todas juntas
   - Total combinaciones: 74
   - Memoria inicial: 463.6 MB


Procesando combinaciones:   0%|          | 0/74 [00:00<?, ?it/s]


🎯 PRODUCTO: Pants
   🌐 Procesando todas las regiones juntas (1/74 - 1.4%)
   Procesando: Pants - None
   📊 Filas antes filtrado: 115,105
   🗑️ Outliers removidos: 302 (0.3%)
   R² ajuste sinusoidal: 0.687
   ✅ Generadas 1,050 predicciones

🎯 PRODUCTO: Puzzles
   🌐 Procesando todas las regiones juntas (2/74 - 2.7%)
   Procesando: Puzzles - None
   📊 Filas antes filtrado: 70,102
   🗑️ Outliers removidos: 124 (0.2%)
   R² ajuste sinusoidal: 0.432
   ✅ Generadas 1,050 predicciones

🎯 PRODUCTO: Face
   🌐 Procesando todas las regiones juntas (3/74 - 4.1%)
   Procesando: Face - None
   📊 Filas antes filtrado: 128,712
   🗑️ Outliers removidos: 913 (0.7%)
   R² ajuste sinusoidal: 0.671
   ✅ Generadas 1,050 predicciones

🎯 PRODUCTO: Dining
   🌐 Procesando todas las regiones juntas (4/74 - 5.4%)
   Procesando: Dining - None
   📊 Filas antes filtrado: 104,308
   🗑️ Outliers removidos: 806 (0.8%)
   R² ajuste sinusoidal: 0.412
   ✅ Generadas 1,050 predicciones

🎯 PRODUCTO: Mystery
   🌐 Procesando 

In [9]:
#ordenamos los datos para que sean iguales que en kaggle
store = dict_results.keys()
total_sales = dict_results.values()

sub = pd.DataFrame({
    "STORE_SUBGROUP_DATE_ID": store,
    "TOTAL_SALES": total_sales
})
test_store_ordenado = pd.read_csv('../data/ids_test.csv')
test_store_ordenado = test_store_ordenado.join(sub.set_index('STORE_SUBGROUP_DATE_ID'), on='STORE_SUBGROUP_DATE_ID', how='left') 
print(test_store_ordenado[test_store_ordenado['TOTAL_SALES'].isna()])
test_store_ordenado.to_csv('../data/submission_sin_region_codigo_pred_total_sales.csv', index=False) 
#est_store_ordenado

Empty DataFrame
Columns: [STORE_SUBGROUP_DATE_ID, TOTAL_SALES]
Index: []
